## Recursive Utility Application

Prepared for WAMS 2017 by [John Stachurski](http://johnstachurski.net)

Let's re-run the `wams_demo` code using parallel programming.

#### Preliminaries

In [1]:
addprocs(4)
nprocs()

5

In [2]:
@everywhere using QuantEcon


Function to compute spectral radius of a matrix.

In [3]:
@everywhere compute_spec_rad(Q::Matrix) = maximum(abs, eigvals(Q))

#### Modeling EZ Preferences


Epstein-Zin utility specification


\begin{equation*}
    V_t = \left[ 
            \zeta C_t^{1-1/\psi}
            + \beta \left\{ R_t \left(V_{t+1}
            \right) \right\}^{1-1/\psi} 
          \right]^{1/(1-1/\psi)}
\end{equation*}


where

\begin{equation*}
    R_t(V_{t+1}) 
    := ( \mathbb{E}_t  V^{1-\gamma}_{t+1} )^{1/(1-\gamma)}
\end{equation*}


* $\beta  \in (0, 1) =$ time discount factor
* $\gamma$ governs the level of relative risk aversion 
* $\psi =$ elasticity of intertemporal substitution

In [4]:

@everywhere mutable struct EpsteinZin{T <: AbstractFloat}
    ψ::T   # Elasticity of intertemporal substitution
    γ::T   # Risk aversion parameter
    β::T   # Time discount factor
    ζ::T   # Preference factor, current consumption
    θ::T   # Derived parameter
end



@everywhere function EpsteinZinBY(; ψ=1.5, γ=10.0, β=0.998)
    ζ = 1 - β
    θ = (1 - γ) / (1 - 1/ψ)
    return EpsteinZin(ψ, γ, β, ζ, θ) 
end

#### Models of Consumption Growth


Discretizing the SV model using two iterations of Rouwenhorst.

The model is 

$$    z' = ρ z + s_z σ e' $$

$$   (σ^2)' = v σ^2 + d + s_σ w' $$

where $\{e\}$ and $\{w\}$ are IID and $N(0, 1)$.  


In [5]:
@everywhere mutable struct StochasticVolatility{T <: AbstractFloat}
    ρ::T
    s_z::T
    v::T
    d::T
    s_σ::T
end

In [6]:
@everywhere function StochasticVolatilityBY(; ρ=0.979, 
                                  s_z=0.044,      
                                  v=0.987,       
                                  σ_bar=0.0078,   
                                  s_σ=2.3e-6)     
    d = σ_bar^2 * (1 - v)
    return StochasticVolatility(ρ, s_z, v, d, s_σ)
end


Discretize the SV model.  Returns a (2, M) matrix 
x_states, each element x of which is a pair (z, σ) stacked vertically, 
and a transition matrix Q such that 

    Q[m, mp] = probability of transition x_states[m] -> x_states[mp]

The strategy is to 

1. Discretize the σ process to produce state values σ_1, ..., σ_I

2. For each σ_i, 

    * discretize the z process to get $z_{i1}, ... z_{iJ}$

In each case, discretization uses Rouwenhorst's method 

The final states are constructed as 

    x_m = (z_{ij}, σ_i), where m = (i - 1) * J + j.
    
Each x_m vector is stacked as a column of x_states.  The transition
probability Q[m, n] from x_m to x_n is computed from the transition matrices
arising from the discretization of σ and z discussed above.



In [7]:
@everywhere multi_from_single(m, J) = div(m - 1, J) + 1, rem(m - 1, J) + 1
@everywhere single_from_multi(i, j, J) = (i - 1) * J + j

In [8]:
@everywhere function discretize_sv(sv::StochasticVolatility, 
                        I::Integer, 
                        J::Integer; 
                        fail_with_neg_σ=false, 
                        verbose=false) 

    # Unpack names
    ρ, s_z, v, d, s_σ = sv.ρ, sv.s_z, sv.v, sv.d, sv.s_σ

    # Discretize σ first
    mc = rouwenhorst(I, v, s_σ, d)
    sig_Q, sig2 = mc.p, collect(mc.state_values)

    # This gives σ^2 values so now we take the square root
    σ_states = similar(sig2)
    if fail_with_neg_σ == true
        @assert all(sig2 .>= 0) "Discretization failed: negative σ values."
    else
        for i in 1:I
            σ_states[i] = sig2[i] < 0 ? 1e-8 : sqrt(sig2[i])
        end
    end

    # Allocate memory
    M = I * J
    z_states = Array{Float64}(I, J)
    q = Array{Float64}(I, J, J)
    x_states = Array{Float64}(2, M)
    Q = Array{Float64}(M, M)
    
    # Discretize z at each σ_i and record state values for z in z_states.
    # Also, record transition probability from z_states[i, j] to 
    # z_states[i, jp] when σ = σ_i.  Store it as q[i, j, jp].
    for (i, σ) in enumerate(σ_states)
        mc_z = rouwenhorst(J, ρ, s_z * σ, 0.0) 
        for j in 1:J
            z_states[i, j] = mc_z.state_values[j]
            for jp in 1:J
                q[i, j, jp] = mc_z.p[j, jp]  
            end
        end
    end

    # Compute x_states and Q
    for m in 1:M
        i, j = multi_from_single(m, J)
        x_states[:, m] = [z_states[i, j], σ_states[i]]
        for mp in 1:M
            ip, jp = multi_from_single(mp, J)
            Q[m, mp] = sig_Q[i, ip] * q[i, j, jp]
        end
    end

    if verbose == true
        return x_states, Q, z_states, σ_states
    else
        return x_states, Q
    end
end

#### Solving the Bansal--Yaron Modle

In [9]:
@everywhere function compute_K_bansal_yaron(ez::EpsteinZin, 
                                sv::StochasticVolatility;
                                μ=0.0015, 
                                I=12,   # discretization in σ
                                J=10)   # discretization in z for each σ

    # Unpack parameters, allocate memory
    γ, β, θ = ez.γ, ez.β, ez.θ
    M = I * J
    K = Array{Float64}(M, M)

    # Discretize SV process 
    x, Q = discretize_sv(sv, I, J)

    for m in 1:M
        for mp in 1:M
            i, j = multi_from_single(m, J)
            z, σ = x[1, m], x[2, m] 
            a = exp((1 - γ) * (μ + z) + (1 - γ)^2 * σ^2 / 2)
            K[m, mp] =  a * Q[m, mp]
        end
    end
    
    return β^θ * K

end

### Computing Spec Rad --- with and w/o parallel

Now let's compute $r(K)$ for the BY model under a range of parameterizations.  

In [10]:
@everywhere sv_by = StochasticVolatilityBY()

In [11]:
J = 20 # grid size
ψ_vals = collect(linspace(1.25, 2.25, J))           # ψ
μ_vals = collect(linspace(0.0005, 0.01, J));        # μ

#### Without Parallelization

Here's the matrix that we'll populate with spectral radius values, one for each parameter pair.

In [12]:
R = Array{Float64}(J, J);

Let's populate this matrix without parallelization.

In [13]:
@time for i in 1:J
    for j in 1:J
        ez = EpsteinZinBY(ψ=ψ_vals[i])
        @assert ez.θ < 0 "Detected non-negative theta value"
        K = compute_K_bansal_yaron(ez, sv_by, μ=μ_vals[j])
        R[i, j] = compute_spec_rad(K)
    end
end

 14.174470 seconds (89.78 M allocations: 1.849 GiB, 1.28% gc time)


#### With Parallelization

In [14]:
R = SharedArray{Float64}(J, J);

In [25]:
@time @sync @parallel for i in 1:J
    for j in 1:J
        ez = EpsteinZinBY(ψ=ψ_vals[i])
        @assert ez.θ < 0 "Detected non-negative theta value"
        K = compute_K_bansal_yaron(ez, sv_by, μ=μ_vals[j])
        R[i, j] = compute_spec_rad(K)
    end
end

  1.657252 seconds (71.27 k allocations: 3.862 MiB, 0.14% gc time)


4-element Array{Future,1}:
 Future(2, 1, 105, #NULL)
 Future(3, 1, 106, #NULL)
 Future(4, 1, 107, #NULL)
 Future(5, 1, 108, #NULL)